### Onboarding Waitlist 

In [14]:
import pandas as pd
import numpy as np 
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning

In [15]:
onboarding_data = pd.read_csv('onboarding_waitlist.csv', low_memory=False)
registration_data = pd.read_excel('network_register.xlsx', sheet_name='Sheet1')
network_waitlist_data = pd.read_excel('network_waitlist.xlsx', sheet_name='Sheet1')

print(len(registration_data))
print(len(onboarding_data))
print(len(network_waitlist_data))

409
294
82


### Prepare Data and Merge 

In [16]:
# Strip of leading and tailing white space 
onboarding_data["safe_address"] = onboarding_data["safe_address"].str.strip()
registration_data["What is your HOPR safe address?"] = registration_data["What is your HOPR safe address?"].str.strip()
network_waitlist_data["Safe address"] = network_waitlist_data["Safe address"].str.strip()

# Make everything lower case letters
onboarding_data["safe_address"] = [x.lower() for x in onboarding_data["safe_address"]]
registration_data["What is your HOPR safe address?"] = [x.lower() for x in registration_data["What is your HOPR safe address?"]]
network_waitlist_data["Safe address"] = [x.lower() for x in network_waitlist_data["Safe address"]] 

# Check for duplicates
dataDup_onboarding = onboarding_data.duplicated(subset=['safe_address'], keep='last')  
dataDup_registration = registration_data.duplicated(subset=['What is your HOPR safe address?'], keep='last')  

# dataDup.value_counts()
onboarding_data['Duplicate'] = dataDup_onboarding
registration_data['Duplicate'] = dataDup_registration 

# Only keep unique values 
onboarding_data_01 = onboarding_data.loc[onboarding_data['Duplicate'] == False]
registration_data_01 = registration_data.loc[registration_data['Duplicate'] == False]

print(onboarding_data_01['Duplicate'].value_counts())
print(registration_data_01['Duplicate'].value_counts())

Duplicate
False    294
Name: count, dtype: int64
Duplicate
False    388
Name: count, dtype: int64


In [17]:
## Delete people who are already in the waitlist from the registration data

x = list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"]) 
print(len(x))

# Drop Avado users in original list (now no problem as only a few have avado it does not iterfer with organic) 
registration_data_01['waitlist_ind'] = np.where(registration_data_01["What is your HOPR safe address?"].isin(x), "YES", "No")
registration_data_02 = registration_data_01[registration_data_01['waitlist_ind'] == "No"]
print(len(registration_data_02))

30
358


In [18]:
waitlist = registration_data_02.merge(onboarding_data_01, how='left', left_on='What is your HOPR safe address?'
                                                    , right_on='safe_address')

waitlist = waitlist[['Time', 'deployment_date', 'What is your Node address', 'safe_address', 'deployment_tx_hash', 'wxHOPR_balance', 'nr_nft']]

# rename comumns 
waitlist = waitlist.rename(columns={"Time": "registration_time", "What is your Node address": "node_address"})
print(len(waitlist))

# exlude non elidgible nodes 
waitlist_01 = waitlist[~waitlist['safe_address'].isnull()] 
print(len(waitlist_01))
display(waitlist_01.head())

358
169


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft
21,2023-09-09T13:27:09.282Z,2023-09-09 12:23:55.000 UTC,0x7ca010ee624b8186e98f41ec2fa6fa329700c104,0xc9f55edb61a23567302e8553f1e5e37574c71e30,0xe76040b340d5afbced1fab05309c64ac710601a3c1a6...,31809.579505,False
40,2023-09-09T14:31:55.044Z,2023-09-09 14:22:30.000 UTC,0x775175769897c0a46781b423c599f1b6a2b4cde8,0x244454ab29bafb5bbb95f8f6de28fcc45e81b620,0x506226a43ddfffc0bfb48124e7b782232624430010b1...,52186.000000,False
46,2023-09-09T14:54:40.369Z,2023-09-09 14:42:15.000 UTC,0x68cff38fa5d50b13734b1e8d7f26df7ef3bb3f69,0x4ab0895c38380dc61b4553963a84033837cd7b6a,0x8371753b265ab7f3dea55c531a06cdd34bffb5ea5fea...,75000.000000,False
52,2023-09-09T15:09:31.289Z,2023-09-09 15:02:25.000 UTC,0x557eae5968432ca52244f16bd364308777b49c63,0xc3fcab887a3f9b07776a47a2fbe452934cecac13,0xcd13c24468d36d45ad353156b8956a9b0b598e8d1372...,70000.000000,False
66,2023-09-09T15:50:12.825Z,2023-09-09 15:36:55.000 UTC,0x27d559ef0a9ab0d00dedbe5e0d5c215ec475ee24,0x4e4c24299bece3aff08b013d10023eb011394199,0xab810a4bac3fda9f2f5f6fa51818cdf0a7f9ad264216...,0.000037,False


### Delete non-elidgible nodes 

In [19]:
# Not enough stake 
waitlist_02 = waitlist_01[waitlist_01['wxHOPR_balance'] >= 10000]
print(len(waitlist_02))
print(waitlist_02.dtypes)

167
registration_time      object
deployment_date        object
node_address           object
safe_address           object
deployment_tx_hash     object
wxHOPR_balance        float64
nr_nft                 object
dtype: object


In [20]:
waitlist_02.loc[:, 'nr_nft'] = waitlist_02['nr_nft'].astype(bool)
print(waitlist_02['nr_nft'].value_counts())

nr_nft
True     113
False     54
Name: count, dtype: int64


In [21]:
# Users with a stake >= 30000 are eligible regardless of NR NFT
waitlist_03 = waitlist_02.copy() 
waitlist_03.loc[:, 'eligible'] = np.where(waitlist_02.loc[:,'wxHOPR_balance'] >= 30000, True, False)

# Users with a stake < 30000 are eligible if they have a network registry NFT
waitlist_04 = waitlist_03.copy()
waitlist_04.loc[:, 'eligible'] = np.where((waitlist_03.loc[:,'wxHOPR_balance'] < 30000) & (waitlist_03.loc[:,'nr_nft'] == True), True, waitlist_03.loc[:, 'eligible'])

print(waitlist_04.loc[:, 'eligible'].value_counts())


eligible
True     164
False      3
Name: count, dtype: int64


In [22]:
waitlist_05 = waitlist_04[waitlist_04.loc[:,'eligible'] == True]
print(len(waitlist_05))

164


In [23]:
# delete users that registered with their peer_id
valid_address = waitlist_05['node_address'].str.startswith('0x')
waitlist_06 = waitlist_05[valid_address]
print(len(waitlist_06))

157


### Sort Users 

In [24]:
nr_waitlist = waitlist_06[waitlist_06['nr_nft'] == True] 
stake_waitlist =waitlist_06[waitlist_06['nr_nft'] == False]

print(len(nr_waitlist))
print(len(stake_waitlist))

107
50


In [25]:
nr_waitlist_01 = nr_waitlist.sort_values(by="deployment_date", ascending=True).reset_index(drop=True)
print(len(nr_waitlist_01))
display(nr_waitlist_01.head())

107


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-13T15:23:47.379Z,2023-09-03 18:30:20.000 UTC,0xc4dd520a3ec78c54f7b4c7853ab9676d3c4a8ae5,0x8609ae79a0c93389a42d7996a9b41faae2137fa1,0x9de18e070a23826bc24d95d635c5e8c810b028636ea3...,29835.143442,True,True
1,2023-09-18T17:17:49.052Z,2023-09-03 19:05:50.000 UTC,0xe4236f6f97ab2369c6ed04f89b7ef807227ab4e1,0x9b4947ef0882ba7d2b854df25601f53c3a0e3113,0x5ade52ebfb44aa67b7a59ed36a20aaf8e6568baaf0b1...,44075.148091,True,True
2,2023-09-19T05:20:10.348Z,2023-09-05 05:26:10.000 UTC,0xa3c372301f1abe885d93abbaef364686ab78b29f,0xb8ce7d9028becffbf613e6c600e9308574c61b0c,0xaf6f4b01fbb05e078358e9df16fac616014e422156ea...,10003.000000,True,True
3,2023-09-14T16:34:08.884Z,2023-09-05 16:15:05.000 UTC,0xf564623c5fdf3a1572d476c68d957884131f040b,0xca017abeb831abe643d6026e956ab2555eb28a4e,0xd17b2fe18c93bb030147de8a6b043da921657d643154...,75000.000000,True,True
4,2023-09-16T10:19:20.287Z,2023-09-06 18:50:10.000 UTC,0x219261727a638b4cdc2d1f9fd9b046a5dc4bb412,0x4ac08ab98a9eb26ba09eca5f2fcf1a2e5465dcfe,0x7c0ec46b791314801ea0cb6739a95a6d6d6e09f9ba2f...,10000.000000,True,True


In [26]:
stake_waitlist_01 = stake_waitlist.sort_values(by="wxHOPR_balance", ascending=False).reset_index(drop=True)
print(len(stake_waitlist))
display(stake_waitlist_01.head())

50


,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-18T10:17:57.736Z,2023-09-17 19:16:10.000 UTC,0x7e5e53560e1b8f08118e76d53f38e022aee25862,0x0c3c8dee94eaf03179fa6e0c8d0834ededdf5b1d,0x5a9af3f59b90a4a5599a51d4eeb51d91e5da44965ae7...,108747.291194,False,True
1,2023-09-15T15:34:11.998Z,2023-09-09 15:24:40.000 UTC,0x6255cf0d5e00dad58fee6a5980cb78fa8d99e083,0xc0926bf77600aca713219507ea1e94dae25067a7,0x2fba447218886f5537418c586572006a0244c44cf656...,80000.000000,False,True
2,2023-09-17T17:47:14.867Z,2023-09-17 17:38:35.000 UTC,0x51b9ad10fe0233f720547bef315c574b72cdd6d6,0x6cdcbe95c7f30a399a37eec8e235ecd6f23118b2,0xdaa7f4cb72d597d28af359d3a4c4947015323125c225...,77536.000000,False,True
3,2023-09-17T18:42:30.003Z,2023-09-17 18:39:05.000 UTC,0xa6f569bcf3a0c772afeb22c49879d4b44462b21a,0x757c5dfeb1e824995a4697f713ff3ab3db5219af,0xbc6f743593855134f63c2b60f2fa6abec6ff106705c4...,77536.000000,False,True
4,2023-09-17T16:57:41.677Z,2023-09-17 16:52:40.000 UTC,0x87a317622368c26b124daf8511ef5393cb002979,0x47480dde4b04eaedf785afacdb929efdd61d6ff6,0x2ceb4f4a688b356b982558b1458d7ff850e6a169b610...,77536.000000,False,True


In [27]:
# Assuming you have two dataframes, nr_waitlist_01 and stake_waitlist_01
nr_chunk_size = 20
stake_chunk_size = 10

nr_index = 0  # Starting index for nr_waitlist_01
stake_index = 0  # Starting index for stake_waitlist_01

final_chunks = []  # List to store concatenated chunks

while nr_index < len(nr_waitlist_01) and stake_index < len(stake_waitlist_01):
    nr_chunk = nr_waitlist_01.iloc[nr_index:nr_index + nr_chunk_size]
    stake_chunk = stake_waitlist_01.iloc[stake_index:stake_index + stake_chunk_size]

    final_chunks.extend([nr_chunk, stake_chunk])
    
    nr_index += nr_chunk_size
    stake_index += stake_chunk_size

# Concatenate the final chunks row-wise
final_waitlist = pd.concat(final_chunks, ignore_index=True)

display(final_waitlist.head())

,registration_time,deployment_date,node_address,safe_address,deployment_tx_hash,wxHOPR_balance,nr_nft,eligible
0,2023-09-13T15:23:47.379Z,2023-09-03 18:30:20.000 UTC,0xc4dd520a3ec78c54f7b4c7853ab9676d3c4a8ae5,0x8609ae79a0c93389a42d7996a9b41faae2137fa1,0x9de18e070a23826bc24d95d635c5e8c810b028636ea3...,29835.143442,True,True
1,2023-09-18T17:17:49.052Z,2023-09-03 19:05:50.000 UTC,0xe4236f6f97ab2369c6ed04f89b7ef807227ab4e1,0x9b4947ef0882ba7d2b854df25601f53c3a0e3113,0x5ade52ebfb44aa67b7a59ed36a20aaf8e6568baaf0b1...,44075.148091,True,True
2,2023-09-19T05:20:10.348Z,2023-09-05 05:26:10.000 UTC,0xa3c372301f1abe885d93abbaef364686ab78b29f,0xb8ce7d9028becffbf613e6c600e9308574c61b0c,0xaf6f4b01fbb05e078358e9df16fac616014e422156ea...,10003.000000,True,True
3,2023-09-14T16:34:08.884Z,2023-09-05 16:15:05.000 UTC,0xf564623c5fdf3a1572d476c68d957884131f040b,0xca017abeb831abe643d6026e956ab2555eb28a4e,0xd17b2fe18c93bb030147de8a6b043da921657d643154...,75000.000000,True,True
4,2023-09-16T10:19:20.287Z,2023-09-06 18:50:10.000 UTC,0x219261727a638b4cdc2d1f9fd9b046a5dc4bb412,0x4ac08ab98a9eb26ba09eca5f2fcf1a2e5465dcfe,0x7c0ec46b791314801ea0cb6739a95a6d6d6e09f9ba2f...,10000.000000,True,True


In [33]:
### sanity checks 

# check users who already got access are not in the waitlist 
print(pd.Series(final_waitlist['safe_address']).isin(list(network_waitlist_data[network_waitlist_data['Eligibility'] == "yes"]["Safe address"])).value_counts())

# check that no duplicates are in the waitlist 
print((final_waitlist.duplicated(subset=['safe_address'], keep='last')).value_counts())

safe_address
False    150
Name: count, dtype: int64
False    150
Name: count, dtype: int64


### Save final waitlist 

In [28]:
final_waitlist.to_excel('final_waitlist.xlsx', index=False)